In [7]:
import os, csv, requests
from bs4 import BeautifulSoup

In [8]:
folderPath = './Testdata/'
fileList = os.listdir(folderPath)
csvName = ""
for f in fileList:
    csvName = f

In [9]:
def hasCode(soup, postId):
    code = None
    answer = soup.find(id="answer-"+postId) #first check if the answer still exists (might be removed)
    
    if answer: #then check for a code example in the answer
        fullAnswer = answer.find('div', class_="s-prose js-post-body")
        code = fullAnswer.findAll('pre') #pre tag is only for code blocks
        if code:
            return code
        else:
            return None
    else:
        return None

In [10]:
def is_c_or_cpp(soup, postId):
    answer = dict()
    c_or_cpp = False
    question = soup.find('div', class_='question')
    post_tags = question.find('div',class_='post-taglist').findAll('a', class_='post-tag')
    
    for p in post_tags:
        if p.text == 'c++' or p.text =='c':
            answer[postId] = p.text
            c_or_cpp = True
            break
            
    if c_or_cpp:
        return answer
    else:
        return None

In [11]:
def line_check(code):
    length = code.count('\n')
    if length>5:
        return True
    else:
        return False

In [12]:
with open(folderPath + csvName, newline='') as file:
    csvFile = csv.DictReader(file)
    for c in range(160): # skips 160 rows
        next(csvFile) 
    count = 1
    countAll = 1
    for row in csvFile:
        fileDict = dict(row)
        if fileDict['PostTypeId'] == '2':
            postId = fileDict['PostId']
            #postId = '951046'
            URL = "https://stackoverflow.com/a/"+postId
            page = requests.get(URL)
            print('opening: ' + URL + "    - #" + str(countAll))
            countAll = countAll + 1
            soup = BeautifulSoup(page.content, 'html.parser')
            
            #check if page still exists
            pageFound = soup.find('div', itemprop="mainEntity")
            if pageFound:
                #check if answer is contains a code example
                code = hasCode(soup,postId)
                if code == None:
                    continue
                else:
                    #check if question is related to c or c++ 
                    answerDict = is_c_or_cpp(soup,postId)
                    if answerDict == None:
                        continue
                    else:
                        #output the anwer into c/c++ files
                        print(postId + " contains c or c++ code in it's answer - #" + str(count))
                        count = count + 1
                        i = 1
                        for c in code:
                            if line_check(c.text)==True:
                                output_path = './output'
                                file_name ="post-"+postId
                                if i>1:
                                    file_name = file_name + "-"+str(i)
                                if answerDict[postId] == "c++":
                                    file_name = file_name +".cpp"
                                elif answerDict[postId] == "c":
                                    file_name = file_name +".c"
                                output = os.path.join(output_path, file_name)
                                file1 = open(output, "w")
                                file1.write(c.text)
                                print(file_name+ " created")
                                file1.close()
                                i = i + 1               

opening: https://stackoverflow.com/a/1197221    - #1
1197221 contains c or c++ code in it's answer - #1
post-1197221.cpp created
post-1197221-2.cpp created
opening: https://stackoverflow.com/a/1198520    - #2
opening: https://stackoverflow.com/a/1198595    - #3
1198595 contains c or c++ code in it's answer - #2
post-1198595.cpp created
opening: https://stackoverflow.com/a/1200005    - #4
opening: https://stackoverflow.com/a/1205932    - #5
opening: https://stackoverflow.com/a/1217804    - #6
opening: https://stackoverflow.com/a/1240906    - #7
opening: https://stackoverflow.com/a/1252441    - #8
opening: https://stackoverflow.com/a/1252443    - #9
opening: https://stackoverflow.com/a/1252456    - #10
opening: https://stackoverflow.com/a/1252478    - #11
opening: https://stackoverflow.com/a/1252606    - #12
opening: https://stackoverflow.com/a/1282482    - #13
opening: https://stackoverflow.com/a/1307110    - #14
opening: https://stackoverflow.com/a/1352728    - #15
opening: https://sta

KeyboardInterrupt: 